In [2]:
import os
import gym
import gym_donkeycar
import numpy as np
import random
import tensorflow
import shimmy
from rich.live import Live
import stable_baselines3
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import SAC

In [14]:
DONKEY_GYM = True
DONKEY_SIM_PATH = "remote"
DONKEY_GYM_ENV_NAME = "donkey-generated-track-v0"
body_style = "car01"
body_rgb = (256, 256, 256)
car_name = "SAC"
font_size = 100
WEB_CONTROL_PORT = int(os.getenv("WEB_CONTROL_PORT", 8887))
port = 9091

In [15]:
folder = '/mnt/d/donkeycar_model/SAC/'

In [20]:
conf = { "DONKEY_GYM":DONKEY_GYM, "DONKEY_SIM_PATH":DONKEY_SIM_PATH, "DONKEY_GYM_ENV_NAME":DONKEY_GYM_ENV_NAME, 
         "body_style":body_style, "body_rgb":body_rgb, "car_name":car_name, "font_size":font_size, 
         "WEB_CONTROL_PORT" : WEB_CONTROL_PORT, "port" : port }

In [30]:
env = gym.make("donkey-minimonaco-track-v0", conf=conf)

INFO:gym_donkeycar.core.client:connecting to localhost:9091 
INFO:gym_donkeycar.envs.donkey_sim:on need car config
INFO:gym_donkeycar.envs.donkey_sim:sending car config.
INFO:gym_donkeycar.envs.donkey_sim:sim started!


starting DonkeyGym env


In [27]:
for i in range(1000):
    # preparation
    if i == 0:
        model = model = SAC("CnnPolicy", env, n_steps=500, verbose=2, buffer_size = 10000 )
        Log={"TestReward":[]}

    # training
    obs = env.reset()
    model.learn(total_timesteps=2500, progress_bar=0)
    model.save(f'{folder}{(i+1)*2500}.pth')

    # testing
    tem = 0
    for _ in range(20):
        obs = env.reset()
        while True:
            action, _states = model.predict(obs, deterministic=True)
            obs, reward, done, info = env.step(action)
            # print (reward, done)
            env.render()
            tem += reward
            if done: break
    Log["TestReward"].append(tem/20)
    print(f"testing {(i+1)*2500} : {tem/20}")
    np.save(f"{folder}/Log_SAC-{(i+1)*2500}.npy", Log)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 41.8     |
|    ep_rew_mean     | -98.2    |
| time/              |          |
|    episodes        | 4        |
|    fps             | 8        |
|    time_elapsed    | 19       |
|    total_timesteps | 167      |
| train/             |          |
|    actor_loss      | 1.49     |
|    critic_loss     | 404      |
|    ent_coef        | 0.981    |
|    ent_coef_loss   | -0.0657  |
|    learning_rate   | 0.0003   |
|    n_updates       | 66       |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 32.4     |
|    ep_rew_mean     | -87.5    |
| time/              |          |
|    episodes        | 8        |
|    fps             | 7        |
|    time_elapsed    | 34       |
|    total_tim

KeyboardInterrupt: 

In [32]:
num = 60000
test_model = SAC.load(f"{folder}{num}.pth")
obs = env.reset()
tem = 0
while True:
    action, _states = test_model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    print (reward, done)
    env.render()
    tem += reward
    if done: break
print(tem)

-9.997999881183421 False
-9.996000152035482 False
-9.854118551967503 False
-9.723778498957023 False
-9.589448718110159 False
-9.449362727423035 False
-9.305723451499004 False
-9.156529189897409 False
-9.07512729287515 False
-9.066228245503046 False
-9.000141878928549 False
-8.95777128524288 False
-8.946254297002923 False
-8.907509639094803 False
-8.883541432318944 False
1.2201436614217895 False
1.3557197846857774 False
1.4567977998590664 False
1.4774242078844708 False
1.4611105726702935 False
1.4610632984178449 False
1.4626435441681227 False
1.4380660084652295 False
1.41542868806644 False
1.3934026708788727 False
1.3876224191659858 False
1.497775255528299 False
1.550786392545643 False
1.6591230410784954 False
1.71399400779885 False
1.7917622831730304 False
1.7637140883502633 False
1.8599478633040711 False
2.0241738316293914 False
2.0623484644052668 False
2.077420768836985 False
2.156161050173886 False
2.152554322879959 False
2.235333302152224 False
2.2653853220386235 False
2.2966150124

In [28]:
env.close()